<div style="display: flex; align-items: center;">
  
  <div style="text-align: left;">
   <h2 style="font-size: 1.8em; margin-bottom: 0;"><b>Going downhill with purpose</b></h2>
   <br>
   <h3 style=" font-size: 1.2em;margin-bottom: 0;">An overview of Gradient Descent</h3>
   <h3 style="font-size: 1.2em; margin-bottom: 0; color: blue;"><i>Dr. Satadisha Saha Bhowmick</i></h3>
  </div>

  <div style="margin-right: 10px;"> 
    <img src="media/images/dsi-logo-600.png" align="right" alt="UC-DSI" scale="0.7;">
  </div>

</div>

<!-- ### Learning Loop -->

<div style="display: flex; align-items: center;gap: 5px;">
  <div style="flex: 1;">
    <h3>About Me</h3>
  <h4>Satadisha Saha Bhowmick, Ph.D</h4>

  <div class="fragment"  style="font-size: 14px;">
    <h4>Affiliation</h4>
    <ul>
      <li>Postdoctoral Teaching Fellow <br> Data Science Institute, University of Chicago</li>
    </ul>
  </div>

  <div class="fragment"  style="font-size: 14px;">
    <h4>Courses I teach</h4>
    <ul>
      <li>Introduction to Data Science</li>
      <li>Mathematical Methods for Data Science</li>
      <li>Ethics, Fairness, Responsibility, and Privacy in Data Science</li>
      <li>Object Oriented Programming with Java</li>
    </ul>
  </div>

  <div class="fragment"  style="font-size: 14px;">
    <h4>Research Interest</h4>
    <ul>
      <li>Information Extraction</li>
      <li>Short Text Mining</li>
    </ul>
  </div>
  
  </div>
  <div style="flex: 1;">
    <img src="media/images/satadisha-photo.png" alt="Self" scale="0.3">
  </div>
</div>


In [1]:
from graphviz import Digraph
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from matplotlib.colors import ListedColormap
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401
from ipywidgets import interact

### Teaching Demonstration
Course Module from CS 231 Machine Learning Fundamentals

<div style="display: flex; gap: 2px;">

  <div style="flex: 1;">
  <h4>Today's Learning Outcomes</h4>
  <ul>
    <li class="fragment"> How do we make models that learn well from data?</li>
    <li class="fragment"> First principles to build intuition on model optimization</li>
    <li class="fragment">Gradient Descent</li>
    <li class="fragment">Impact of Learning Rate</li>
    <li class="fragment">Gradient Descent in practice</li>
  </ul>

  
  </div>

  <div style="flex: 1;">
  <h4>Prerequisites</h4>
  <ul>
    <li class="fragment"> Vector Calculus</li>
    <li class="fragment"> Linear Algebra</li>
  </ul>
  </div>

</div>

### Teaching Demonstration
More to Follow

  <ul>
    <li>Learning Rate Schedules</li>
    <li>Strategies for Learning Rate adjustment</li>
      <ul>
        <li>AdaGrad</li>
        <li>Newton's Method</li>
      </ul>
    <li>Algorithmic variants for Gradient Descent</li>
  </ul>

<!-- ### Learning Loop -->

<div style="display: flex; align-items: center;gap: 20px;">
  <div style="flex: 3;">
    <h3>Learning Loop</h3>
    <p>Machine Learning systems within a supervised setup typically involve a <b>model of choice</b> and a <b>loss function</b>.</p>

  <ul>
    <li class="fragment">The model is a (mathematical) function of its parameters.</li>
    <li class="fragment">The loss function measures how far an estimated value of an example is from its true value as a function of the model parameters.</li>
    <ul>
      <li class="fragment">Common loss functions: Squared Loss, Log Loss, Cross-Entropy Loss (for multiclass classification)</li>
    </ul>
  </ul>

  
<span class="fragment"> In some problem settings, we have a general *objective* function instead of a loss, like the *likelihood* of a certain output or label given the data, that we might want to maximize.</span>

<span class="fragment"> The purpose of training in supervised learning is to obtain the parameter values for the best possible model. 
<span style="color:blue;">The *best* model involves parameters that can lead to ***optimizing*** the training loss.</span></span>
  
  </div>
  <div style="flex: 1;">
    <img src="media/images/learning-loop.png" alt="The Learning Loop" scale="1">
  </div>
</div>


### Model Optimization

<span class="fragment"> The optimal training loss is when the loss function assumes its lowest value. Optimization is <span style="color:blue;"><b>minimizing the loss</b></span>!</span>

<span class="fragment"> In most cases, it is impossible or difficult to find the minimum of the loss function analytically; instead, we turn to numerical methods from the field of <i>continuous optimization</i>.</span>

<span class="fragment"> In this lecture, we will mostly talk about <b>unconstrained optimization</b>.</span>

### Setting the Scene

We begin by setting some key terminologies.

<span class="fragment"> Supervised learning involves training data consisting of many examples. Let a single datapoint or example be denoted by: $(x,y) \text{ where } x \in {\cal R}^{n}$.
</span>

<span class="fragment"> Depending on the problem $y$ could be:</span>
<ul>
  <li class="fragment"> Numeric: $y \in {\cal R}$ for regression problems</li>
  <li class="fragment">Categorical: for classification problems</li>
</ul>

<span class="fragment">The model output $\hat{y}$ is a function of its features. For a regression problem: $\hat{y} = f_{\theta}(x)$, where $f_{\theta}: {\cal R}^{n} \rightarrow {\cal R}$ and  $ \theta \in {\cal R}^{n} $</span>

<span class="fragment">A training set of $m$ examples, $m_{train}: {(x^{(1)}, y^{(1)}), (x^{(2)}, y^{(2)}) \dots (x^{(m)}, y^{(m)})}$</span>

<span class="fragment">Loss is calculated on individual examples. For a regression problem, we will typically use squared loss : $\mathcal{L}(\theta) = (y-\hat{y})^2$</span>

<span class="fragment">To learn from the entire training set, we aggregate the loss from each example into a **Cost function** : $$\mathcal{J}(\theta) = \frac{1}{m} \sum_{i=1}^{m} (y^{(i)} - \hat{y}^{(i)})^2$$</span>

### The Loss Function

Our loss function $f_{\theta}$ should be:
<ul>
    <li class="fragment"> Continuous : well defined and limit exists, no breaks!</li>
    <li class="fragment"> Differentiable : derivative exists at every point of its domain.</li>
    <li class="fragment"> Convex : any line segment connecting two points of the graph of $f_{\theta}$ lies on or above the graph</li>
</ul>

#### Univariate functions
| Convex Function | Non-convex Function |
|:-------:|:-------:|
| <img src="media/images/parabola.png" width="300"> | <img src="media/images/non-convex.png" width="300"> |

<br><br>

<span class="fragment">
<h4 style="color: blue; font-weight: bold; font-style: italic;">Over to You &#9997;</h4>
Can you identify the minimum and maximum points of each of these functions? Is there only one per curve?</br>
</span>

<span class="fragment">
<p style="color: blue;">Convex functions have one <b>global minima</b>!</p>
</span>

#### Multivariate functions
| Convex Function | Non-convex Function |
|:-------:|:-------:|
| <img src="media/images/convex-multi.png" width="800"> | <img src="media/images/non-convex-multi.png" width="900"> |

### How to detect optimal points of a function?

Let's start with the univariate case.

<span class="fragment">
  <p><b>At a local optima:</b></p>
  <ol>
    <li>The tangent line is horizontal.
        <ul>
        <li>First-order derivative (gradient) is 0.</li>
        </ul>
    </li>
    <li>Function curvature is indicative.
        <ul>
        <li>Second-order derivative is positive at a minima and negative at a maxima.</li>
        </ul>
    </li>
  </ol>
</span>

### How to detect optimal points of a function?

<div style="display: flex; align-items: center; gap: 5px;">

  <div class="fragment"; style="flex: 2;">
  <p>For $f(x) = x^4 + 7x^3 + 5x^2 - 17x + 3$</p>
  <ol>
    <li> $f^{'}(x) = 4x^3 + 21x^2 + 10x - 17$
        <ul>
        <li>finding the optimal points becomes a root finding problem</li>
        <li>optimal points at  -4.48, -1.43 and 0.66 are real of the equation $f^{'}(x)=0$</li>
        </ul>
    </li>
    <li> $f^{''}(x) = 12x^2 + 42x + 10$
        <ul>
        <li>plugging in the roots of $f^{'}(x)$ we get two local minima at -4.48 and 0.66  as well as a local maxima at -1.43</li>
        </ul>
    </li>
  </ol>
  </div>
  <div class="fragment"; style="flex: 1;">
    <img src="media/gif/local_optima_animation.gif" alt="Optimal Points of a Function" scale="0.6;" style="width: 90%;">
    <img src="media/images/roots.png" alt="Optimal Points of a Function" scale="0.6;" style="width: 90%;">
  </div>
</div>

<!-- SINGLE-COLUMN SECTION -->
<div class="fragment" style="margin-top: 20px;">
  <p>Around a local minima, $f(x)$ is decreasing if $x$ is less than the minima and increasing if it is greater. These behaviours hold uniformly for all $x$ values for a convex function with a single global minima.</p>
</div>

### How to detect optimal points of a multivariate function?

For $f(x): {\cal R}^{n} \rightarrow {\cal R}$, we still have the same derivative tests for a local optima $x^*$.

<span class="fragment" style="font-size: 14px;">
  <ol>
    <li>Gradient is a $n$ dimensional vector of first-order partial derivatives.
      $\nabla f(x) = \begin{pmatrix}
      \frac{\partial f}{\partial x_1} \\ \frac{\partial f}{\partial x_2} \\ \vdots \\ \frac{\partial f}{\partial x_n}
      \end{pmatrix} = 0$
    </li>
    <li>Second-order partial derivatives are organized in a $n \times n$ matrix called the Hessian:
      $$
      H(f)(\mathbf{x}) = \nabla^{2} f(x)
      \begin{bmatrix}
      \frac{\partial^2 f}{\partial x_1^2} 
      & \frac{\partial^2 f}{\partial x_1 \partial x_2} 
      & \cdots
      & \frac{\partial^2 f}{\partial x_1 \partial x_n}
      \\[0.5em]
      \frac{\partial^2 f}{\partial x_2 \partial x_1} 
      & \frac{\partial^2 f}{\partial x_2^2} 
      & \cdots
      & \frac{\partial^2 f}{\partial x_2 \partial x_n}
      \\[0.5em]
      \vdots & \vdots & \ddots & \vdots
      \\[0.5em]
      \frac{\partial^2 f}{\partial x_n \partial x_1} 
      & \frac{\partial^2 f}{\partial x_n \partial x_2} 
      & \cdots 
      & \frac{\partial^2 f}{\partial x_n^2}
      \end{bmatrix}
      $$
      Assuming that the matrix is symmetric and invertible at a critical point,
        <ul>
        <li>At a local minima, $H(x^*)$ is <b>positive definite</b>. All eigenvalues must be positive ($\forall v; v^{T}Hv > 0$).</li>
        <li>At a local maxima, $H(x^*)$ is <b>negative definite</b>. All eigenvalues must be negative ($\forall v; v^{T}Hv < 0$).</li>
        </ul>
    </li>
  </ol>

  You can find more about the <a href="https://en.wikipedia.org/wiki/Second_partial_derivative_test" target="_blank">second partial derivative test</a> as well as the simplified determinant test for the bivariate case.
</span>

### Big Idea

<div style="display: flex; align-items: center; gap: 5px;">

  <div class="fragment"; style="flex: 1;">
    <p>Loss surfaces for most ML algorithms involve complex, high-dimensional functions. We also do not know much about the form they take.
    Analytically deriving closed form optimal solutions here becomes infeasible.</p>
    <p><b>Iterative Approach</b>: For a <b>convex loss function</b>, start from any given point and "<i>walk down</i>" the curve in small steps to reach the deepest point.</p>
    <span class="fragment">
    <p><b>For a multivariate $f(\theta): {\cal R}^{n} \rightarrow {\cal R}$, how should we proceed for this?</b></p>
    <ul>
      <li>Look at Taylor Approximations!</li>
      <li>Knowing local behaviour gives global information.</li>
    </ul>
    </span>
  </div>

  <div class="fragment"; style="flex: 1;">
    <img src="media/gif/taylor.gif" alt="Taylor approximations" scale="0.45;" style="width: 90%;">
  </div>
</div>

<div class="fragment">
$$
f(\theta + h) \approx f(\theta) + f^{'}(\theta)h + \frac{f^{''}(\theta)}{2!}h^2 + \frac{f^{'''}(\theta)}{3!}h^3 + \dots
$$
</div>

### Gradient Descent from first principles

Starting from point $\vec{\theta}$, we wish to take a small step $\vec{h}$, such that  $f(\vec{\theta}+\vec{h})<f(\vec{\theta})$.

If $\vec{h} \in {\cal R}^{n}$ is small enough, i.e. $\lVert \vec{h} \rVert \approx 0$, we can use Taylor expansion to obtain a locally linear approximation of $f$ around $\vec{\theta} = (\theta_{1}, \theta_{2} \dots \theta_{n})$.

$$
f(\vec{\theta}+\vec{h}) \approx f(\vec{\theta}) + \nabla f(\vec{\theta})^{T} \vec{h}
$$

This assumes that the function $f$ is differentiable around $\vec{\theta}$ and $\nabla f$ is non-zero. If we choose $\vec{h} = - \eta \nabla f$ for a small positive factor $\eta$, then as per the approximation

$$\begin{aligned}
&& f(\vec{\theta} - \eta \nabla f) \approx f(\vec{\theta}) - \eta \nabla f(\vec{\theta})^{T} \nabla f(\vec{\theta}) \\
&& f(\vec{\theta} - \eta \nabla f) \approx f(\vec{\theta}) - \eta \lVert \nabla f(\vec{\theta}) \rVert^{2}_{2}
\end{aligned}$$

As we know the norm of a vector to be positive, subtracting $\eta \lVert \nabla f(\vec{\theta}) \rVert^{2}_{2}$ from $f(\vec{\theta})$ would result in a decrease of the value of $f$.

### Gradient Descent

<div class="fragment">

Intuitively, gradient $\nabla f$ is known as the direction of steepest increase of $f$. Hence taking a step proportional to $-\nabla f$ will be in the direction of the steepest reduction for $f$ and take us to its minima. 

The magnitude of the step is determined by the (positive) factor $\eta$, also known as the *learning rate* or *step-size*.

This is the crux of *Gradient Descent*.
</div>

<div class="fragment" style="
    border: 4px solid #3b82f6;
    background: #eef6ff;
    padding: 10px 15px;
    border-radius: 6px;
    margin: 10px auto;
    width: 90%;
    box-sizing: border-box;">

  <b>Definition.</b>  
  $\color{blue}{\textbf{Gradient Descent}}$ is an iterative algorithm to minimize a convex loss function. Starting from an initial value, the parameter vector $\vec{\theta}$ is updated in each iteration, as per the following rule, until a convergence criterion is reached:
  $$
  \vec{\theta} \leftarrow \vec{\theta} - \eta \nabla f
  $$
</div>


### Gradient Descent Pseudocode

<div style="
    border: 2px solid #a2aab9ff;
    background: #eef3ff;
    padding: 14px;
	font-size: 15px;
    border-radius: 6px;
    font-family: 'Courier New', monospace;
    width: 90%;
    margin: 15px auto;
    box-sizing: border-box;">

<pre style="
    font-size: 1.5em !important;        /* ←  MAKE TEXT BIGGER */
    line-height: 1.4em;
">

function GRADIENT_DESCENT(J, v, η):
    θ⁰ ← v
    while not converged:
        grad ← ∇J(θ⁽ᵗ⁾)
	∀i: θᵢ⁽ᵗ⁺¹⁾ ← θᵢ⁽ᵗ⁾ − η * gradᵢ
    return θ

</pre>

</div>

### Gradient Descent: Example 1
<div style="display: flex; align-items: center; gap: 5px;">

  <div class="fragment"; style="flex: 1;">
  <p>For the convex function $f(\theta_{1}, \theta_{2}) = \theta_{1}^2+\theta_{2}^2$</p>

  <span class="fragment">
  $$
  \begin{aligned}
    \frac{\partial f}{\partial \theta_{1}} = 2\theta_{1}; \frac{\partial f}{\partial \theta_{2}} = 2\theta_{2} \\
  \end{aligned}
  $$
  </span>

  <span class="fragment">
  Starting at $\theta^{(0)} = (0.8, 0.8)$ with a step size of $\eta=0.1$, here we run the Gradient Descent algorithm for 20 steps.

  The value of $f$ at $\theta^{(20)}$ after 20 iterations is $1.7014 \times 10^{-4}$, which is quite close to the global minimum of 0.
  </span>
  </div>

  <div class="fragment"; style="flex: 1;">
    <img src="media/gif/gd_fixed_0.10.gif" alt="Gradient Descent Example" scale="0.55;" style="width: 90%;">
  </div>
</div>

### Gradient Descent over Cost Functions

**Linearity of Gradient** implies that $\nabla (f_{1}+f_{2}) = \nabla f_{1}+\nabla f_{2}$. This property has practical importance.

Implies that, the gradient of the entire cost function $\mathcal{J}$ can be found by taking the sum (average) of the gradient of the loss $f$ on individual examples.


### Convergence

In an iterative algorithm like Gradient Descent, convergence can be defined by a few possible stopping criteria and some pre-defined tolerance value $\tau > 0$.

<span class="fragment">1. Maximum number of iterations: <i>use this regardless</i></span>

<span class="fragment">2. $\lVert\mathcal{J}(\theta^{t+1})-\mathcal{J}(\theta^{t})\rVert < \tau$</span>

<span class="fragment">3. $\lVert\theta^{t+1}-\theta^{t}\rVert < \tau$</span>

<span class="fragment">4. $\lVert \nabla f(\theta^{t+1}) \rVert < \tau$</span>

<div style="display: flex; gap: 30px; align-items: flex-start; min-height: 400px;">

  <!-- LEFT COLUMN -->
  <div style="flex: 1;">

### Choosing a good step-size

<span class="fragment">The parameter learning rate $\eta$ determines the magnitude of the step taken in the direction of negative gradient.</span>

<span class="fragment">

For example, consider gradient descent on a function $f(x) = (x-5)^2$ starting at $x^{(0)}=10$, with learning rate $\eta=1$. The minima is at $x=5$. Is this a good learning rate?
</span>

<span class="fragment">
<h4 style="color: blue; font-weight: bold; font-style: italic;">Over to You &#9997;</h4>

What values do $x$ assume by the end of the first and second iterations of gradient descent: $x^{(1)}$ and $x^{(2)}$?<br>
What does that tell us about the learning rate?
</span>

  </div>

  <!-- RIGHT COLUMN -->
  <div style="flex: 1; display: flex; flex-direction: column; align-items: center;">

  <!-- IMAGE FIRST -->
  <div class="fragment">
      <img src="media/gif/gd_oscillation.gif" alt="Gradient Descent Oscillation" style="width: 90%;">
  </div>

  <!-- TEXT ON NEXT CLICK -->
  <div class="fragment" style="margin-top: 20px; text-align: center;">
      Learning rate ideally should not be too large, else Gradient Descent may never converge!
  </div>

  </div>

</div>


### Choosing a good step-size

<div class="fragment">
  Learning rate is important for convergence. But how do we choose it?
</div>

<div class="fragment">

  Consider the convex function $f(x) = \frac{1}{2}x^2$ with a global minima at $x=0$.
  Start with $\eta^{(0)} = 0.1$, $x^{(0)}=1$, and use a decreasing sequence of learning rates.  
  Do we reach the minima?
</div>

<div class="fragment">
  <ol>
    <li>$\eta^{0}=0.1*1,\ \eta^{1}=0.1*\tfrac12,\ \eta^{2}=0.1*\tfrac14,\dots, \eta^{t}=0.1*\tfrac1{2^t}$</li>
    <li>$\eta^{0}=0.1*1,\ \eta^{1}=0.1*\tfrac12,\ \eta^{2}=0.1*\tfrac13,\dots, \eta^{t}=0.1*\tfrac1{t+1}$</li>
  </ol>
</div>

<div class="fragment">

With $\nabla f(x) = x$, we have the following Gradient Descent updates,
$$\small \begin{aligned}
x^1 = x^0 - \eta^{0}x^0 = (1-\eta^{0})x^0 \\
x^2 = (1-\eta^{1})x^1 = (1-\eta^{1})(1-\eta^{0})x^0 \\
x^t = (1-\eta^{t-1})\dots(1-\eta^{1})(1-\eta^{0})x^0 = x^{(0)} \prod_{k=0}^{t-1} (1-\eta^{k})
\end{aligned}$$
As $t \rightarrow \infty$, $x^{\infty} = x^{(0)} \prod_{k=0}^{\infty} (1-\eta^{k})$
</div>


### Choosing a good step-size

<div class="fragment">

Convergence to $x=0$ as $t \rightarrow \infty$, depends on $\prod_{k=0}^{\infty} (1-\eta^{k}) \rightarrow 0$.<br>
For $0 \leq \eta^{k} < 1$, we know from the <a href= "https://archive.org/details/theoryandapplica031692mbp" target="_blank"> criteria of convergence for infinite products</a>:
$$\small \begin{aligned}
\text{if }\sum_{k=0}^{\infty}\eta^{k}={\infty}  \text{ ; } \prod_{k=0}^{\infty} (1-\eta^{k})=0\\
\text{if } \sum_{k=0}^{\infty}\eta^{k}<{\infty} \text{ ; } \prod_{k=0}^{\infty} (1-\eta^{k})=\text{C; a finite positive number }
\end{aligned}$$
</div>

<div class="fragment">

<table style="border-collapse: collapse; width: 80%; margin: auto;">
  <tr>
    <th style="border: 1px solid black; text-align: center;">Case 1</th>
    <th style="border: 1px solid black; text-align: center;">Case 2</th>
  </tr>

  <tr>
    <td style="border: 1px solid black; text-align: center;">
    $\eta^{0}=0.1*1,\ \eta^{1}=0.1*\tfrac12,\ \eta^{2}=0.1*\tfrac14,\dots, \eta^{t}=0.1*\tfrac1{2^t}$
    </td>
    <td style="border: 1px solid black; text-align: center;">
    $\eta^{0}=0.1*1,\ \eta^{1}=0.1*\tfrac12,\ \eta^{2}=0.1*\tfrac13,\dots, \eta^{t}=0.1*\tfrac1{t+1}$
    </td>
  </tr>

  <tr>
    <td style="border: 1px solid black; text-align: center;">
      $\sum_{k=0}^{\infty}\eta^{k} = \sum_{k=0}^{\infty}\frac{0.1}{2^k}$
    </td>
    <td style="border: 1px solid black; text-align: center;">
      $\sum_{k=0}^{\infty}\eta^{k} = \sum_{k=0}^{\infty}\frac{0.1}{k+1}$
    </td>
  </tr>

  <tr>
    <td style="border: 1px solid black; text-align: center;">
      $\sum_{k=0}^{\infty}\eta^{k} = 0.2$
    </td>
    <td style="border: 1px solid black; text-align: center;">
      $\sum_{k=0}^{\infty}\eta^{k} = \infty$
    </td>
  </tr>

  <tr>
    <td style="border: 1px solid black; text-align: center;">
      As $t \rightarrow \infty$, $x^{\infty} = x^{(0)} C \neq 0$
    </td>
    <td style="border: 1px solid black; text-align: center;">
      As $t \rightarrow \infty$, $x^{\infty} = x^{(0)} 0 = 0$
    </td>
  </tr>

  <tr>
    <td style="border: 1px solid black; text-align: center;">
      finite movement; freezes above minimum
    </td>
    <td style="border: 1px solid black; text-align: center;">
      infinite movement; eventually converges to minimum
    </td>
  </tr>
</table>

</div>

### Choosing a good step-size

<div class="fragment">
  Could be tuned like a hyperparameter! Remember:
  <ol>
    <li>Cannot be too high when gradients are large.</li>
    <li>Cannot be too low when on a shallow curve — Gradient Descent slows down as it gets closer to the minima.</li>
    <li>Ideally, should go down as we run more iterations and get closer to the minima.</li>
  </ol>
</div>

### Algorithmic Variants

<span class="fragment">
<b>Batch Gradient Descent</b>

- Regular Gradient Descent involves computing the full cost function – loss function for all training examples – to adjust a parameter value in a single iterative step. Computationally infeasible for large datasets!
</span>

<span class="fragment">
<b>Stochastic Gradient Descent</b>

- Every step of gradient descent is taken based of the loss computed on one example. 
- Extremely fast updates. But calculations based on single example can introduce a lot of variance (random updates hence stochastic)! It also never fully converges but hovers asymptotically close to the local minima.
- Updates are noisy but the randomness helps us get out of local minima or other bad regions. Less prone to overfitting.
</span>

<span class="fragment">
<b>Mini Batch Gradient Descent</b>

- Randomly split training data into mini-batches, compute a gradient descent step according to a mini-batch. Most commonly used in practice.
- Parallelizable on GPUs.
- A happy medium. Smoother convergence but also computationally less expensive.
</span>

### What next?

Follow-up lecture:

  <ul>
    <li>Learning Rate Schedules</li>
      <ul>
        <li>Can we decreasing sequence of learning rates?</li>
      </ul>
    <li>Strategies for Learning Rate adjustment</li>
      <ul>
        <li>AdaGrad: asymmetrical learning rate along different dimensions</li>
        <li>Newton's Method: use curvature information from second order derivative instead of fixed learning rate</li>
      </ul>
  </ul>

### Adagrad

<div class="fragment">
Should all features be assigned the same learning rate?

- Even after feature scaling, different features have different rates of influence on the learning problem, and consequently on the model and the cost function.
</div>

<div class="fragment">
Solution:

- Keep a history of feature updates
- Decay the learning rate of a feature over time in proportion to the update history.
    - Large gradients: Dense features or high rate of change; small learning rate.
    - Small gradients: Sparse features or low rate of change; large learning rate.
</div>

#### Adagrad Pseudocode

<div style="
    border: 2px solid #a2aab9ff;
    background: #eef3ff;
    padding: 14px;
    border-radius: 6px;
    font-family: 'Courier New', monospace;
    width: 90%;
    margin: 15px auto;
    box-sizing: border-box;">

<pre>
function ADAGRAD(J, v, η):
    ∀i: θ⁽⁰⁾ ← v; grad_hᵢ⁽⁰⁾ ← 0
    while not converged:
        ∀i: grad_hᵢ⁽ᵗ⁾ ← grad_hᵢ⁽ᵗ⁻¹⁾+ ∇Jᵢ(θᵢ⁽ᵗ⁾)²
        ∀i: θᵢ⁽ᵗ⁺¹⁾ ← θᵢ⁽ᵗ⁾ − η/√(grad_hᵢ⁽ᵗ⁾ + ε) * ∇Jᵢ(θᵢ⁽ᵗ⁾)
    return θ
</pre>
convergence is defined by $\lVert\theta^{t+1}-\theta^{t}\rVert_2 < \tau$
</div>

<div style="display: flex; gap: 30px; align-items: flex-start; min-height: 400px;">

  <!-- LEFT COLUMN -->
  <div style="flex: 1;">

  ### Newton's Method

  <div class="fragment">

  Gradient Descent assumes a linear approximation of a function and moves down it's slope in small steps to iteratively reach the minima. Alternatively, one can approach optimization using a quadratic approximation of the loss function.
  </div>

  <div class="fragment">

  If $\vec{h} \in {\cal R}^{n}$ is small enough, i.e. $\lVert \vec{h} \rVert \approx 0$, Taylor expansion can be used to obtain a quadratic approximation of $f$ around $\vec{\theta} = (\theta_{1}, \theta_{2} \dots \theta_{n})$ to capture curvature, not just slope.
  $$
  f(\vec{\theta}+\vec{h}) \approx f(\vec{\theta}) + \nabla f(\vec{\theta})^{T} \vec{h} + \frac{1}{2} \vec{h}^T H(\vec{\theta})\vec{h}
  $$
  </div>

  </div>

  <!-- RIGHT COLUMN -->
  <div style="flex: 1; display: flex; flex-direction: column; align-items: center;">

  <!-- IMAGE FIRST -->
  <div class="fragment">
      <img src="media/images/taylor_2.png" alt="Taylor Expansion" style="width: 90%;">
  </div>
  
  </div>

</div>

<div class="fragment">
  This is equivalent to locally fitting a parabola around the point of interest $\vec{\theta}$. 
  Here we assume that the function is <b>twice differentiable</b>. The iterative search here follows by taking the next step to the minima of this convex parabola.
</div>

### Newton's Method

<div class="fragment">

Differentiating the equation for $f(\vec{\theta}+\vec{h})$ w.r.t. $\vec{h}$ we have,
$$
\nabla_{\vec{h}} [f(\vec{\theta}) + \nabla f(\vec{\theta})^{T} \vec{h} + \frac{1}{2} \vec{h}^T H(\vec{\theta})\vec{h}] = \nabla f(\vec{\theta}) + H(\vec{\theta})\vec{h}
$$
</div>

<div class="fragment">

Setting the derivative to 0 we can derive,
$$\begin{aligned}
&& \nabla f(\vec{\theta}) + H(\vec{\theta})\vec{h} = 0 \\
&& \vec{h} = -  H^{-1}(\vec{\theta})\nabla f(\vec{\theta})
\end{aligned}$$

This is the update that is made in each iteration of the Newton's method until convergence. Note that we no longer rely on a scalar learning rate.
$$
\vec{\theta} \leftarrow \vec{\theta} -  H^{-1}(\vec{\theta})\nabla f(\vec{\theta})
$$
</div>

### Newton's Method: Example 2
<div style="display: flex; align-items: center; gap: 5px;">

  <div class="fragment"; style="flex: 1;">

  <p>For the convex function $f(\theta_{1}, \theta_{2}) = \theta_{1}^2+\theta_{2}^2$</p>

<!--
  <span class="fragment">
  $$
  \begin{aligned}
    \frac{\partial f}{\partial \theta_{1}} = 2\theta_{1}; \frac{\partial f}{\partial \theta_{2}} = 2\theta_{2} \\
  \end{aligned}
  $$
  </span>
-->

  <span class="fragment">
  Starting at $\theta^{(0)} = (0.8, 0.8)$, here we run Newton's Method.

  After just 5 iterations, value of $f$ at $\theta^{(5)}$ is $1.7014 \times 10^{-4}$. This is quite close to the global minimum of 0.

  It took Gradient Descent 20 iterations to get to this value.
  </span>
  </div>

  <div class="fragment"; style="flex: 2;">
    <img src="media/gif/newton.gif" alt="Gradient Descent" scale="0.55;" style="width: 90%;">
  </div>
</div>

### Newton's Method

<span class="fragment">
<p> Pros: </p>
<ul>
    <li>Newton's method ensures much faster convergence than Gradient Descent.</li>
    <li>Automatically adapts the step size based on the curvature of the function (Hessian), removing the need for hyperparameter tuning of learning rate.</li>
</ul>
</span>
<span class="fragment">
<p> Cons: </p>
<ul>
    <li>Computing both the Gradient and the Hessian and its inverse involves high computational costs for very high dimensional function</li>
    <li>Starting from a bad initial guess, may cause the function to diverge or oscillate over the optima.</li>
</ul>
</span>

### Where do we start?

<div class="fragment">
<p>Gradient Descent requires us to initialize the parameter search with an initial guess. What constitutes a good guess?<p>
</div>

<div class="fragment">

**Random Initialization**
- Small random values.
- Things are better with a convex objective function.
    - Unique global minima, so guaranteed to converge.
- For non-convex function, we can get trapped in a local minima depending on initialization.
    - Start the search from different positions, and see where each search ends up.
    - Choose the best out of all minimizers.
</div>

### Algorithmic Variants

<span class="fragment">
<b>Batch Gradient Descent</b>

- Regular Gradient Descent involves computing the full cost function – loss function for all training examples – to adjust a parameter value in a single iterative step. Computationally infeasible for large datasets!
</span>

<span class="fragment">
<b>Stochastic Gradient Descent</b>

- Every step of gradient descent is taken based of the loss computed on one example. 
- Extremely fast updates. But calculations based on single example can introduce a lot of variance (random updates hence stochastic)! It also never fully converges but hovers asymptotically close to the local minima.
- Updates are noisy but the randomness helps us get out of local minima or other bad regions. Less prone to overfitting.
</span>

<span class="fragment">
<b>Mini Batch Gradient Descent</b>

- Randomly split training data into mini-batches, compute a gradient descent step according to a mini-batch. Most commonly used in practice.
- Parallelizable on GPUs.
- A happy medium. Smoother convergence but also computationally less expensive.
</span>

### Gradient Descent in Practice

<div style="display: flex; align-items: center; gap: 5px;">

  <div class="fragment"; style="flex: 1;">

  <div>
  <b>Problem</b>: Binary Classification of Tumor Cell as Benign or Malignant
  </div>

  <div>
  <b>Dataset</b>: Wisconsin Diagnostic Breast Cancer (WDBC) dataset; <br>
           569 cell samples; 8 real valued features
  </div>

  <div>
  <b>Train/Test Split</b>: 80/20
  </div>

  <div>
  <b>Model of Choice</b>: Logistic Regression
  </div>

  <div>
  <b>Loss Function</b>: Logistic Loss
  </div>

  </div>



  <div class="fragment"; style="flex: 2;">
    <img src="media/images/model_comparison.png" alt="Gradient Descent" scale="0.40;" style="width: 90%;">
  </div>
</div>

### Gradient Descent in Practice

<div class="fragment">

<table style="border-collapse: collapse; width: 80%; margin: auto;">
  <tr>
    <th style="border: 1px solid black; text-align: center;">Optimization Method</th>
    <th style="border: 1px solid black; text-align: center;">Train Accuracy</th>
    <th style="border: 1px solid black; text-align: center;">Test Accuracy</th>
    <th style="border: 1px solid black; text-align: center;">Test F1 Score</th>
  </tr>

  <tr>
    <td style="border: 1px solid black; text-align: center;">
    Gradient Descent
    </td>
    <td style="border: 1px solid black; text-align: center;">
    0.9824175824175824
    </td>
    <td style="border: 1px solid black; text-align: center;">
    <b>0.9912280701754386</b>
    </td>
    <td style="border: 1px solid black; text-align: center;">
    <b>0.993006993006993</b>
    </td>
  </tr>

  <tr>
    <td style="border: 1px solid black; text-align: center;">
    Stochastic Gradient Descent
    </td>
    <td style="border: 1px solid black; text-align: center;">
    0.9758241758241758
    </td>
    <td style="border: 1px solid black; text-align: center;">
    0.9736842105263158
    </td>
    <td style="border: 1px solid black; text-align: center;">
    0.9790209790209791
    </td>
  </tr>

  <tr>
    <td style="border: 1px solid black; text-align: center;">
    Mini-Batch Gradient Descent
    </td>
    <td style="border: 1px solid black; text-align: center;">
    0.9868131868131869
    </td>
    <td style="border: 1px solid black; text-align: center;">
    0.9824561403508771
    </td>
    <td style="border: 1px solid black; text-align: center;">
    0.9859154929577465
    </td>
  </tr>

  <tr>
    <td style="border: 1px solid black; text-align: center;">
    AdaGrad
    </td>
    <td style="border: 1px solid black; text-align: center;">
    0.9868131868131869
    </td>
    <td style="border: 1px solid black; text-align: center;">
    0.9824561403508771
    </td>
    <td style="border: 1px solid black; text-align: center;">
    0.9859154929577465
    </td>
  </tr>

  <tr>
    <td style="border: 1px solid black; text-align: center;">
    Newton's Method
    </td>
    <td style="border: 1px solid black; text-align: center;">
    <b>0.989010989010989</b>
    </td>
    <td style="border: 1px solid black; text-align: center;">
    0.9649122807017544, 
    </td>
    <td style="border: 1px solid black; text-align: center;">
    0.9714285714285714
    </td>
  </tr>
</table>

</div>

### Thank You!

<div style="display: flex; align-items: center; gap: 5px;">

  <div style="flex: 1;">
  <h3>Remember:</h3>
  <h4>Things going downhill, not so bad!</h4><br><br>
  <h4>The gradients are the friends you made along the way!</h4>

  </div>



  <div style="flex: 2;">
    <img src="media/images/gd-cartoon.png" alt="Gradient Descent" scale="0.40;" style="width: 90%;">
  </div>
</div>